In [1]:
from tensorflow_addons.layers import inalu

import tensorflow as tf
import numpy as np
import os
import ast
import abc
import random
from typing import List, Dict, Union, Tuple
import numpy as np
from scipy.stats import truncnorm
import argparse
import timeit
from datetime import datetime
%load_ext tensorboard

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # disable cuda sepeed up
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # disable CPU wornings


2022-10-23 23:47:46.936945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 23:47:46.936973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-o", "--output", dest="output", default="naly_syn_simple_arith")
parser.add_argument("-d", "--dist", dest="dist", default="normal", help="Prob.Dist")
parser.add_argument("-p", "--params",dest="params" , default="(-3,3)", type=ast.literal_eval)
parser.add_argument("-e", "--ext",dest="ext" , default="(10,15)", type=ast.literal_eval)

parser.add_argument("-n", "--nalu", dest="nalu", default="nalui1")
parser.add_argument("-se", "--seed", dest="seed", default=42, type=int)
parser.add_argument("-op", "--operation", dest="op", default="MUL")



args = parser.parse_args([])


def sample(dist, params, numDim = 2, numDP = 64000):
    data = np.zeros(shape=(numDP, numDim))
    if dist == "normal":
        intmean = (params[0] + params[1]) / 2
        intstd = (params[1] - params[0]) / 6
        print(
            "Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tmean(s)\t %s\n\t\tdata <\t %s\n\t\tstd \t %s" % (
                dist, params[0], intmean, params[1], intstd))
        mi, ma = (params[0] - intmean) / intstd, (params[1] - intmean) / intstd
        data = np.reshape(truncnorm.rvs(mi, ma, intmean, intstd, size=numDim * numDP), data.shape)

    elif dist == "uniform":
        print("Generating Data: \nInt: \tdist \t %s\n\t\tdata >=\t %s\n\t\tdata <\t %s\n\t\t" % (
        dist, params[0], params[1]))
        data = np.reshape(np.random.uniform(params[0], params[1], size=numDim * numDP), data.shape)
    elif dist == "exponential":
        data = np.random.exponential(params, size=(numDP, numDim))
    else:
        raise Exception("Unknown distribution")
    data = np.reshape(data, [-1])  # reshape to mix both distributions per instance!
    np.random.shuffle(data)
    data = np.reshape(data, (numDP, numDim))
    return data


def operation(op, a, b):
    if op.lower() == "mul":
        return a * b
    if op.lower() == "add":
        return a + b
    if op.lower() == "sub":
        return a - b
    if op.lower() == "div":
        return a / b

random.seed(args.seed)
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

data = sample(args.dist, args.params)
lbls = operation(args.op, data[:,0], data[:,1])
lbls = np.reshape(lbls, newshape=(-1, 1))

int_data = sample(args.dist, args.params)
int_lbls = operation(args.op, int_data[:,0], int_data[:,1])
int_lbls = np.reshape(int_lbls, newshape=(-1, 1))

ext_data = sample(args.dist, args.ext)
ext_lbls = operation(args.op, ext_data[:,0], ext_data[:,1])
ext_lbls = np.reshape(ext_lbls, newshape=(-1, 1))

Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 -3
		mean(s)	 0.0
		data <	 3
		std 	 1.0
Generating Data: 
Int: 	dist 	 normal
		data >=	 10
		mean(s)	 12.5
		data <	 15
		std 	 0.8333333333333334


In [3]:
BATCH_SIZE = 64

data_dp = tf.data.Dataset.from_tensor_slices((data, lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
int_data_dp = tf.data.Dataset.from_tensor_slices((int_data, int_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)
ext_data_dp = tf.data.Dataset.from_tensor_slices((ext_data, ext_lbls)).prefetch(tf.data.AUTOTUNE).batch(BATCH_SIZE)

2022-10-23 23:48:11.044704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-23 23:48:11.044769: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-23 23:48:11.044808: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (filip-HP-ProBook-440-G3): /proc/driver/nvidia/version does not exist
2022-10-23 23:48:11.045266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:

from abc import abstractmethod



class NALURegularizer(tf.keras.regularizers.Regularizer):
  def __init__(self, reg_coef=0.1):
    self.reg_coef = reg_coef

  def __call__(self, var: List[tf.Variable]) -> tf.Tensor:
    return self.reg_coef * tf.reduce_mean(tf.math.maximum(tf.math.minimum(-var, var) + 20, 0))

  def get_config(self):
    return {'reg_coef': float(self.reg_coef)}


class NALUInterface(metaclass=abc.ABCMeta):
    @classmethod
    def __subclasshook__(cls, subclass):
        return (
            hasattr(subclass, 'get_gates_variables') and
            callable(subclass.get_gates_variables) or
            NotImplemented)
    
    @abc.abstractclassmethod
    def get_gates_variables(self) -> List[tf.Variable]:
        """Return list of tf gating variables"""
        raise NotImplementedError

class NALUModelSuperClass(NALUInterface):
    
    steps_counter = tf.Variable(0, trainable=False)
    epoch_counter = tf.Variable(0, trainable=False)
    reinitialization_counter = tf.Variable(0, trainable=False)
    regularize = tf.Variable(False, trainable=False)

    def get_gates_variables(self) -> List[tf.Variable]:
        return [v for l in self.layers if isinstance(l, NALUInterface) for v in l.get_gates_variables()]

    def get_regularization_loss(self):
        return tf.math.reduce_sum(self.losses)

    @tf.function
    def train_step_active(self, data):
        x, y = data
        with tf.GradientTape(watch_accessed_variables = True) as tape:
            logits = self(x, training=True) 
            loss_value = self.compiled_loss(y, logits)
            tf.math.add(loss_value, tf.cond(
                tf.math.logical_and(self.regularize, tf.math.less(loss_value, 1.0)),
                lambda: self.get_regularization_loss(), 
                lambda: tf.constant(0, dtype = tf.float32)))
        
        grads = tape.gradient(loss_value, tape.watched_variables())
        self.optimizer.apply_gradients(zip(grads, tape.watched_variables()))
        self.compiled_metrics.update_state(y, logits)
        return  {m.name: m.result() for m in self.metrics}

    @tf.function
    def train_step_gating(self, data):
        x, y = data
        with tf.GradientTape(watch_accessed_variables = False) as tape:
            for v in self.gate_var:
                tape.watch(v)
            logits = self(x, training=True) 
            loss_value = self.compiled_loss(y, logits)
            tf.math.add(loss_value, tf.cond(
                tf.math.logical_and(self.regularize, tf.math.less(loss_value, 1.0)),
                lambda: self.get_regularization_loss(), 
                lambda: tf.constant(0, dtype = tf.float32)))

        grads = tape.gradient(loss_value, tape.watched_variables())
        self.optimizer.apply_gradients(zip(grads, tape.watched_variables()))
        self.compiled_metrics.update_state(y, logits)
        return  {m.name: m.result() for m in self.metrics}



class NALUModel(tf.keras.Model, NALUModelSuperClass, NALUInterface):

    def __init__(self, *args, **kwargs):
        super(NALUModel, self).__init__(*args, **kwargs)

        self.layer = inalu.NALU(2,1)
        self.layer2 = inalu.NALU(1,1) #tf.keras.layers.Dense(1)
        self.layer3 = tf.keras.layers.Dense(1, kernel_regularizer="l2")
        self.gate_var = self.get_gates_variables()


    def call(self, inputs: tf.Tensor, training: bool = True):
        return self.layer3(self.layer2(self.layer(inputs)))

    @tf.function
    def train_step(self, data):
        """
        Specifying tf.function(input_signature=...) slows down the computation, but it leads to greater control:
        https://www.neuralconcept.com/post/in-graph-training-loop
        
        """
        print("tracking check")
        
        return tf.cond(
            tf.math.greater(self.steps_counter.assign_add(1), 2), 
            lambda: self.train_step_active(data), 
            lambda: self.train_step_gating(data))
        

In [9]:
mm = NALUModel()
mm.compile(optimizer="adam", loss="mse", metrics=["mae"])


class DelayRegularize(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        regularize_epochs = 10
        tf.cond(tf.math.greater(self.model.epoch_counter.assign_add(1),regularize_epochs), lambda: self.model.regularize.assign(True), lambda: None)
        
metrtics = mm.fit(data_dp, epochs = 3, callbacks=[DelayRegularize()])

Epoch 1/3
tracking check
1000/1000 [==============================] - 6s 3ms/step - loss: 0.9506 - mae: 0.6268
Epoch 2/3
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8483 - mae: 0.5821
Epoch 3/3
1000/1000 [==============================] - 3s 3ms/step - loss: 0.7302 - mae: 0.5523


In [10]:
mm.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.87698233>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.86694765>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99432087>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9172727>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9391996>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9996104>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008183975>]

In [11]:
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp  # <- Name of this `run`
writer = tf.summary.create_file_writer(logdir)
# Initialization
loss_fn = tf.keras.losses.MeanSquaredError()
reg_fn = NALURegularizer()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

x = tf.random.normal(shape=(10, 2))
y = tf.random.normal(shape=(10, 1))
# Start tracing and store it in `tf.summary`
tf.summary.trace_on(graph=True, profiler=False)
# Code: Begin
#############
# Call `tf.function` when tracing.

tmp = mm.train_step_gating(next(iter(data_dp)))

# Code: End
###########
with writer.as_default():
  tf.summary.trace_export(
      name="train_step_trace",  # <- Name of tag
      step=0,
      profiler_outdir=logdir)

In [13]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6007 (pid 31811), started 0:00:02 ago. (Use '!kill 31811' to kill it.)